In [55]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import mne
import matplotlib.pyplot as plt
import pyvista
import ipywidgets
import ipyevents
import pyvistaqt
import yasa

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.utils import class_weight

import tensorflow as tf
from tensorflow.keras import layers, models

In [3]:
%matplotlib qt
# to make plots interactive

## *Pickle data

### Importing data

In [4]:
file_path = r"C:\EEG DATA\FL_label_data.pickle"
# added r in front of file path to make it a raw string, to make sure that \ is not interpreted as a newline character

# open the pickle file
with open(file_path, "rb") as file:
    label_data = pickle.load(file)

# show the label_data type
print(type(label_data))

<class 'dict'>


### Functions

In [5]:
# to return all the results
# returns a dict so should have commas between values

def extract_onsets(label_data):
    onset_dict = {}
    for key, value in label_data.items():
        labels = np.atleast_1d(value['label'])
        onsets = np.atleast_1d(value['onset'])
        # to ensure that labels and onsets are treated as array
        # because subsequently using np.where
        indices = np.where((labels == 1) | (labels == 2))[0]
        # returns indices where the label is 1 (N2) or 2 (N3)
        if indices.size > 0 and np.all(indices < len(onsets)):
            # to ensure that no out-of-bounds error
            selected_onsets = onsets[indices]
            # retrieve onset value corresponding to label 1 or 2
            onset_dict[key] = selected_onsets
            # save extracted onset under correct key in dict
            print(f"Key: {key}, Onset values for labels 1 (N2) and 2 (N3): {', '.join(map(str, selected_onsets))}")
        else:
            print(f"Key: {key}, Warning: The indices do not match")
    return onset_dict
    # returning the onset_dict and what you're printing
    # should I be only returning what is supposed to be printed? or maybe only the dict, since already has commas?

def group_by_increment(onset_values, increment=30):
    groups = []
    # will be a list of lists
    current_group = [float(onset_values[0])]
    # initializes this list with the first value from onset_values (the input)
    
    for i in range(1, len(onset_values)):
        # loops through all the onset values
        if onset_values[i] - onset_values[i - 1] == increment:
            # if i = 1, if onset_values[1] - onset_values[0] == 30
            current_group.append(float(onset_values[i]))
            # add the value at current index
        else:
            # if not a difference of 30
            # means you've reached the end of that sublist
            if len(current_group) > 1:
                # if there is more than one value in that group
                groups.append(current_group)
                # add the sublist to the big list
            current_group = [float(onset_values[i])]
            # starts a new current group with the new value at the current index
    
    if len(current_group) > 1:
        groups.append(current_group)
    # once you exit the group, if the last current_group contains more than one value
    # then you can add it to group
    # to make sure that last sequence is not left out
    
    return groups

def extract_segments(raw, groups):
    raw_segments = []
    
    for group in groups:
        start = group[0]
        # start = first value in group
        #stop = min(group[-1], max_time) 
        stop = group[-1]
        # stop = last value in group

        #if start >= max_time:
            #continue
        # takes the smaller of the two values
        segment = raw.copy().crop(tmin=start, tmax=stop)
        raw_segments.append(segment)
    
    return raw_segments

## *Raw data Participant 067

### Importing 

In [6]:
# follow instructions from YASA

participant_067_file = r"C:\EEG DATA\067\eeg\TMR.vhdr"
participant_067_raw = mne.io.read_raw_brainvision(vhdr_fname=participant_067_file, preload=True)
participant_067_raw.filter(0.1, 40)
# bandpass filter between 0.1 Hz and 40 Hz
participant_067_raw.resample(100)
# downsample to 100 Hz
participant_067_raw.pick(['Fz'])

Extracting parameters from C:\EEG DATA\067\eeg\TMR.vhdr...
Setting channel info structure...
Reading 0 ... 14024599  =      0.000 ... 28049.198 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 16501 samples (33.002 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:   14.2s


<RawBrainVision | TMR.eeg, 1 x 2804920 (28049.2 s), ~21.4 MiB, data loaded>

### Put data in YASA format

In [7]:
participant_067_data, participant_067_times = participant_067_raw.get_data(return_times=True) 

# put data in npz format for the hypnogram 
# numpy array
np.savez("participant_067_npz.npz", data=participant_067_data, times=participant_067_times, ch_names=participant_067_raw.ch_names, sfreq=participant_067_raw.info["sfreq"])

In [8]:
# format the npz data

npzfile_067 = np.load("participant_067_npz.npz")
data_067, ch_names_067 = npzfile_067['data'], npzfile_067['ch_names']
sf_067 = 100
times_067 = np.arange(data_067.size) / sf_067

print(data_067.shape, ch_names_067)
print(np.round(data_067[:, 0:5], 3))
# prints the data from the first 4 channels

# 1 channel with 2,237,708 samples

(1, 2804920) ['Fz']
[[-0. -0. -0.  0.  0.]]


In [9]:
# retrieve the labels for participant 067

original_labels_067 = label_data['067']['label']
original_labels_067

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1,
       2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 1, 1, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2,

In [11]:
# remap values for hypnogram
# original values:
# 0: wake/N1, 1: N2, 2: N3, 3: REM
# YASA values
# 0: wake, 1: N1, 2: N2, 3: N3, 4: REM

yasa_labels_067 = np.array(original_labels_067, dtype=object)
yasa_labels_067[original_labels_067 == 0] = "N1"
# only keep N1 for efficiency
yasa_labels_067[original_labels_067 == 1] = "N2"
yasa_labels_067[original_labels_067 == 2] = "N3"
yasa_labels_067[original_labels_067 == 3] = "REM"

yasa_labels_067

array(['N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1',
       'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1',
       'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1',
       'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1',
       'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1', 'N1',
       'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2',
       'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N3', 'N2',
       'N2', 'N2', 'N3', 'N3', 'N3', 'N3', 'N3', 'N2', 'N2', 'N3', 'N2',
       'N3', 'N3', 'N3', 'N3', 'N3', 'N3', 'N1', 'N1', 'N1', 'N1', 'N2',
       'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N1', 'N1', 'N1', 'N1',
       'N1', 'N1', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2',
       'N1', 'N1', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2',
       'N1', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2', 'N2',
       'N2', 'N2', 'N3', 'N3', 'N3', 'N3', 'N3', 'N

### Hypnogram and spectrogram

In [13]:
hypnogram_067 = yasa.Hypnogram(yasa_labels_067, freq="30s")
hypnogram_067.hypno

Epoch
0      N1
1      N1
2      N1
3      N1
4      N1
       ..
929    N1
930    N1
931    N1
932    N1
933    N1
Name: Stage, Length: 934, dtype: category
Categories (7, object): ['WAKE', 'N1', 'N2', 'N3', 'REM', 'ART', 'UNS']

In [14]:
# plot the hypnogram

fig, ax = plt.subplots(1, 1, figsize=(7,3), constrained_layout=True, dpi=80)
ax = hypnogram_067.plot_hypnogram(fill_color="gainsboro", ax=ax)

In [15]:
### upsample the hypnogram for the spectrogram

hypnogram_067_upsampled = yasa.hypno_upsample_to_data(hypno=hypnogram_067.hypno, sf_hypno=(1/30), data=data_067, sf_data=sf_067)
print(hypnogram_067_upsampled.shape, 'Unique values =', np.unique(hypnogram_067_upsampled))

10-May-25 13:36:51 | WARNING | Hypnogram is SHORTER than data by 29.20 seconds. Padding hypnogram with last value to match data.size.


(2804920,) Unique values = ['N1' 'N2' 'N3' 'REM']


In [16]:
hypnogram_067_upsampled
# the first values and the last values are N1 so this is normal

array(['N1', 'N1', 'N1', ..., 'N1', 'N1', 'N1'], dtype=object)

In [17]:
# convert back to numbers
# had converted to stages before for hypnogram

# remap values for spectrogram
# original values:
# 0: wake/N1, 1: N2, 2: N3, 3: REM
# YASA values
# 0: wake, 1: N1, 2: N2, 3: N3, 4: REM

hypnogram_067_upsampled_int = np.array(hypnogram_067_upsampled)
hypnogram_067_upsampled_int[hypnogram_067_upsampled == "N1"] = 1
# only keep N1 for efficiency
hypnogram_067_upsampled_int[hypnogram_067_upsampled == "N2"] = 2
hypnogram_067_upsampled_int[hypnogram_067_upsampled == "N3"] = 3
hypnogram_067_upsampled_int[hypnogram_067_upsampled == "REM"] = 4

hypnogram_067_upsampled_int

array([1, 1, 1, ..., 1, 1, 1], dtype=object)

In [18]:
fig = yasa.plot_spectrogram(data_067[0, :], sf_067, hypnogram_067_upsampled_int)
# data must be a 1D numpy array
fig.suptitle("Spectrogram with Hypnogram of Participant 067", fontsize=14)

plt.show()

### Sleep spindles detection

In [19]:
# convert back to correct unit
data_067_uv = data_067 * 1e6

# convert back to 1D array data since only one channel

data_067_1d = data_067_uv[0]  

In [20]:
# use yasa spindle detection

spindles_067 = yasa.spindles_detect(data_067_1d, sf_067, ch_names=ch_names_067, hypno=hypnogram_067_upsampled_int, include=(2,3))
spindles_067.summary().round(3)

,Start,Peak,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry,Stage,Channel,IdxChannel
0,1857.69,1858.02,1858.27,0.58,15.563,3.653,1.144,0.352,12.733,6.0,0.559,2,Fz,0
1,1916.07,1916.54,1916.61,0.54,14.545,3.060,0.992,0.277,12.061,6.0,0.855,2,Fz,0
2,1938.52,1939.09,1939.61,1.09,15.338,3.424,0.924,0.258,12.877,13.0,0.518,2,Fz,0
3,1944.09,1944.82,1944.95,0.86,11.038,2.505,0.660,0.295,12.667,11.0,0.839,2,Fz,0
4,1947.26,1947.53,1947.77,0.51,17.510,4.616,1.362,0.304,13.302,7.0,0.519,2,Fz,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,25113.50,25113.76,25114.59,1.09,16.385,3.626,1.041,0.525,13.003,14.0,0.236,2,Fz,0
339,25161.63,25161.84,25162.20,0.57,16.410,3.405,1.047,0.299,13.510,8.0,0.362,2,Fz,0
340,25215.11,25215.77,25216.07,0.96,12.452,3.209,1.059,0.433,13.624,13.0,0.680,2,Fz,0
341,25219.13,25219.49,25219.92,0.79,12.982,2.668,0.875,0.411,12.573,10.0,0.450,2,Fz,0


In [21]:
# visualize it by NREM stage 2 and 3

spindles_067.summary(grp_chan=True, grp_stage=True, aggfunc='mean')

,,Count,Density,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry
Stage,Channel,,,,,,,,,,
2,Fz,317,1.444191,0.738170,14.281003,3.289478,0.988945,0.338512,12.798005,9.227129,0.488861
3,Fz,26,0.346667,0.677308,14.803251,3.373247,1.061472,0.332794,12.914434,8.346154,0.452184


In [22]:
# plot the average spindle

spindles_067.plot_average(errorbar=None, palette="Set1")

<Axes: title={'center': 'Average spindle'}, xlabel='Time (sec)', ylabel='Amplitude (uV)'>

### Define 1-second epochs

In [41]:
#label_data_onsets_067 = label_data_onsets['067']
#groups_067 = group_by_increment(label_data_onsets_067, increment=30)
#groups_067

In [23]:
epoch_length_sec = 1
# desired length of each epoch
n_epochs = int(len(data_067_1d) / (sf_067 * epoch_length_sec))
# to figure out how many one-second epochs fit into the signal
# number of epochs = total samples divided by samples per epoch

# Split the signal into 1-second epochs
epochs_067 = np.array_split(data_067_1d[:n_epochs * sf_067], n_epochs)
# 2,237,708 samples
# 100 samples/sec
# 22,377 of 1-second epochs

# with this code, dropping the last epoch if not one second

In [24]:
epochs_067[:10]

# epochs_067 is an array of 1-second EEG arrays (each 100 samples long)

[array([-0.62835643, -1.31628026, -1.01950265,  1.12225964,  1.55107474,
        -1.78472509, -0.5497783 ,  2.02310686,  0.31905683,  0.32302353,
         2.73213683,  0.52404977,  0.03026184,  3.13428945,  0.06483986,
        -2.08002465, -0.03398247,  0.4364957 , -0.79241005,  2.10529447,
         5.34584362,  3.76891672,  3.47132509,  6.05778272,  5.08407269,
         2.7270536 ,  5.18326168,  6.70540981,  7.84204366,  8.01185821,
         5.51786612,  5.26418984,  3.55946388,  2.82678103,  3.66658441,
         0.42013341,  0.84780429,  4.26790669,  5.0690656 ,  4.47381479,
         5.21441128,  4.28348371, -1.00661914, -0.82441821,  1.07392682,
        -0.6363271 , -0.92030665, -1.30917343, -3.53220739, -4.01275344,
        -0.54645271,  0.46380164, -1.89865715, -1.51370857,  0.385095  ,
        -1.95697037, -3.75468553, -2.04371871, -3.46848665, -5.02046583,
        -3.18375085, -3.90353087, -5.14831716, -3.5074161 , -4.54020269,
        -5.55898135, -3.09534904, -2.92993085, -6.2

In [25]:
len(epochs_067)

28049

In [26]:
data_067_1d[:5]

array([-0.62835643, -1.31628026, -1.01950265,  1.12225964,  1.55107474])

### Label the epochs

We will use as labels 0 for no spindle and 1 for spindle in the 1-second epoch. Importantly, the code checks whether part of the spindle is in that epoch. We are not checking whether the entire spindle is in the epoch.

##### Function

In [27]:
def label_spindle_epochs(epochs, spindle_starts, spindle_ends, epoch_length_sec=1):

    epoch_starts = np.arange(len(epochs)) * epoch_length_sec
    # new np array with the start time of each epoch
    # epoch_starts[i] is the start time of each epoch

    epoch_labels = np.zeros(len(epochs), dtype=int)
    # initialize all the labels as 0 initially

    for start, end in zip(spindle_starts, spindle_ends):
        # loop through the start and end times of detected spindles by YASA
        for i, epoch_start in enumerate(epoch_starts):
            # loop through the one-second epochs that are not labelled yet
            epoch_end = epoch_start + epoch_length_sec
            # for each epoch, calculate the epoch end time
            # which is epoch_start + length of epoch
            # so now have the time range of each epoch
            if (start < epoch_end) and (end > epoch_start):
                # if the spindle started before the epoch ends
                # and the spindle ended after the epoch started
                epoch_labels[i] = 1
                
    return epoch_labels

##### For participant 067

In [28]:
spindles_067_df = spindles_067.summary()

spindle_starts_067 = spindles_067_df['Start'].values
spindle_ends_067 = spindles_067_df['End'].values

epoch_labels_067 = label_spindle_epochs(epochs_067, spindle_starts_067, spindle_ends_067)

In [29]:
epoch_labels_067[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [30]:
len(epoch_labels_067)

28049

### Prepare EEG data for CNN input

CNN expects the data to be: [batch_size, time_steps, channels]
batch_size: number of 1-second epochs
time_steps: number of timepoints per epoch (sampling rate)
channels: number of EEG channels

In [31]:
# reshape the EEG epochs (not the labels)
# currently epochs_067 is a list of 1D arrays

#epochs_067_reshaped = np.array(epochs_067)
#epochs_067_reshaped = epochs_067_reshaped.reshape(-1, epochs_067_reshaped.shape[1], 1)
# reshaped to number of epochs, number of samples per epoch, 1
# -1: numpy calculates the appropriate size based on the other dimensions
# epochs_reshaped.shape[1]: contains information about number of samples in each epoch
#  1: one channel

In [32]:
#epochs_067_reshaped.shape

(28049, 100, 1)

In [47]:
def build_cnn_model(input_shape=(100, 1)):

    # linear embedding layer
    input_layer = tf.keras.layers.Input(shape=input_shape)

    # Three convolutional blocks (like having three pattern detectors)

    # First convolution block, kernel size of 5
    padded1 = tf.keras.layers.ZeroPadding1D(padding=2)(input_layer)
    conv1 = tf.keras.layers.Conv1D(filters=10, kernel_size=5, strides=1, padding='valid')(padded1)
    # each filter learns a different type of short-time feature
    # stride of 1, moves one step at a time
    conv1 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv1)
    conv1 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv1)
    # K = 2
    conv1 = tf.keras.layers.BatchNormalization()(conv1)

    # Second convolution block, kernel size of 11
    padded2 = tf.keras.layers.ZeroPadding1D(padding=5)(input_layer)
    conv2 = tf.keras.layers.Conv1D(filters=10, kernel_size=11, strides=1, padding='valid')(padded2)
    conv2 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv2)
    conv2 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv2)
    conv2 = tf.keras.layers.BatchNormalization()(conv2)

    # Third convolution block, kernel size of 21
    padded3 = tf.keras.layers.ZeroPadding1D(padding=10)(input_layer)
    conv3 = tf.keras.layers.Conv1D(filters=10, kernel_size=21, strides=1, padding='valid')(padded3)
    conv3 = tf.keras.layers.LeakyReLU(alpha=0.01)(conv3)
    conv3 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv3)
    conv3 = tf.keras.layers.BatchNormalization()(conv3)

    # Concatenate the outputs of all blocks
    concatenated = tf.keras.layers.Concatenate()([conv1, conv2, conv3])

    # GRU Layer
    gru = tf.keras.layers.GRU(64)(concatenated)

    # Fully connected (dense) layer
    dense = tf.keras.layers.Dense(64, activation='relu')(gru)
    # add a Dropout layer to prevent overfitting
    dense = tf.keras.layers.Dropout(0.5)(dense)

    # Two softmax outputs for dual-task classification
    #output_task1 = tf.keras.layers.Dense(2, activation='softmax', name='task1')(dense)
    #output_task2 = tf.keras.layers.Dense(2, activation='softmax', name='task2')(dense)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

    # Create the model
    #model = tf.keras.models.Model(inputs=input_layer, outputs=[output_task1, output_task2])
    model = tf.keras.models.Model(inputs=input_layer, outputs=output)

    # Compile the model
    #model.compile(optimizer='adam', loss={'task1': 'categorical_crossentropy', 'task2': 'categorical_crossentropy'}, metrics={'task1': 'accuracy', 'task2': 'accuracy'})
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Return the compiled model
    return model



In [48]:
input_shape = (100, 1)  
cnn_model = build_cnn_model(input_shape)

# look at the architecture of the model
cnn_model.summary()

C:\Users\roseb\anaconda3\envs\msc_research_project\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 100, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_3    │ (None, 104, 1)    │          0 │ input_layer_1[0]… │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_4    │ (None, 110, 1)    │          0 │ input_layer_1[0]… │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding1d_5    │ (None, 120, 1)    │          0 │ input_layer_1[0]… │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 100, 10)   │         60 │ zero_padding1d_3… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 100, 10)   │        120 │ zero_padding1d_4… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 100, 10)   │        220 │ zero_padding1d_5… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_3       │ (None, 100, 10)   │          0 │ conv1d_3[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_4       │ (None, 100, 10)   │          0 │ conv1d_4[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_5       │ (None, 100, 10)   │          0 │ conv1d_5[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_3     │ (None, 50, 10)    │          0 │ leaky_re_lu_3[0]… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_4     │ (None, 50, 10)    │          0 │ leaky_re_lu_4[0]… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_5     │ (None, 50, 10)    │          0 │ leaky_re_lu_5[0]… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 50, 10)    │         40 │ max_pooling1d_3[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 50, 10)    │         40 │ max_pooling1d_4[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 50, 10)    │         40 │ max_pooling1d_5[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 50, 30)    │          0 │ batch_normalizat… │
│ (Concatenate)       │                   │            │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 23,177 (90.54 KB)

 Trainable params: 23,117 (90.30 KB)

 Non-trainable params: 60 (240.00 B)

In [49]:
epochs_067_np = np.array(epochs_067)
print(epochs_067_np.shape)
print(epoch_labels_067.shape)

(28049, 100)
(28049,)


In [50]:
# split into X and y (labels) data
X = epochs_067_np
y = epoch_labels_067

# split into train and test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# stratify = y to ensure that same proportion of classes in both training and test set

In [56]:
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict ={0: class_weights[0], 1: class_weights[1]}

In [57]:
training_info = cnn_model.fit(X_train, y_train, validation_split=0.2, epochs=20, batch_size=64, class_weight=class_weight_dict)

Epoch 1/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.8925 - loss: 0.3879 - val_accuracy: 0.8746 - val_loss: 0.2965
Epoch 2/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.8874 - loss: 0.3043 - val_accuracy: 0.9229 - val_loss: 0.1970
Epoch 3/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.9080 - loss: 0.2629 - val_accuracy: 0.8663 - val_loss: 0.3472
Epoch 4/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.9075 - loss: 0.2402 - val_accuracy: 0.9240 - val_loss: 0.1864
Epoch 5/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.9210 - loss: 0.2083 - val_accuracy: 0.8908 - val_loss: 0.2527
Epoch 6/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.9121 - loss: 0.2181 - val_accuracy: 0.9407 - val_loss: 0.1443
Epoch 7/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.9273 - loss: 0.1851 - val_accuracy: 0.9082 - val_loss: 0.2335
Epoch 8/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.9326 - loss: 0.1695 - val_accu

In [58]:
def plot_training_history(training_info):
  fig, axs = plt.subplots(1, 2, figsize=(16, 5))
  axs[0].plot(training_info.history['loss'], label="training set")
  axs[0].plot(training_info.history['val_loss'], label="validation set")
  axs[0].set_xlabel("Epoch")
  axs[0].set_ylabel("Loss")
  axs[0].grid(True)
  axs[0].legend()
  try:
    axs[1].plot(training_info.history['accuracy'], label="training set")
    axs[1].plot(training_info.history['val_accuracy'], label="validation set")
    axs[1].set_xlabel("Epoch")
    axs[1].set_ylabel("Accuracy")
    axs[1].grid(True)
    axs[1].legend()
  except:
    pass
  plt.show()

plot_training_history(training_info)

In [59]:
cnn_model.evaluate(X_test, y_test)

176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9391 - loss: 0.1487 


[0.151688352227211, 0.9390374422073364]

In [63]:
# for more metrics

y_pred = cnn_model.predict(X_test)
y_pred_labels = (y_pred > 0.5).astype(int)

print(confusion_matrix(y_test, y_pred_labels))
print(classification_report(y_test, y_pred_labels))

176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
[[5173  320]
 [  22   95]]
              precision    recall  f1-score   support

           0       1.00      0.94      0.97      5493
           1       0.23      0.81      0.36       117

    accuracy                           0.94      5610
   macro avg       0.61      0.88      0.66      5610
weighted avg       0.98      0.94      0.96      5610



Precision: of the samples predicted as spindles, how many are correct
Recall: how many actuall spindles are detected
F1-score: balance between precision and recall